In [19]:
import os
import json
import hashlib

# Đường dẫn đến thư mục gốc
root_dir = "D:\\AI\\recommender-systems\\crawl-data-tiki"

# Đường dẫn đến thư mục lưu trữ file JSON
output_dir = "tiki_new"

# Tạo thư mục lưu trữ file JSON nếu chưa tồn tại
os.makedirs(output_dir, exist_ok=True)

In [9]:
# Tạo biến đếm ID người dùng
user_id_counter = 1

# Tạo một danh sách để lưu trữ dữ liệu người dùng
user_data = []

# Tạo một từ điển để lưu trữ thông tin người dùng dựa trên tên
user_dict = {}

# Duyệt qua tất cả các thư mục con trong thư mục gốc
for dir_name in os.listdir(root_dir):
    dir_path = os.path.join(root_dir, dir_name)
    if os.path.isdir(dir_path):
        # Duyệt qua tất cả các tệp tin trong thư mục con
        for file_name in os.listdir(dir_path):
            file_path = os.path.join(dir_path, file_name)
            if file_name.endswith(".json"):
                # Đọc dữ liệu từ tệp tin JSON
                with open(file_path, "r", encoding="utf-8") as f:
                    data = json.load(f)

                # Lấy thông tin người dùng từ dữ liệu
                for review in data["reviews"]:
                    reviewer = review["reviewer"]
                    name = reviewer["name"]

                    # Xử lý trường hợp người dùng đã tồn tại trong từ điển
                    if name in user_dict:
                        # Cập nhật danh mục cho người dùng đã tồn tại
                        if dir_name not in user_dict[name]["categories"]:
                            user_dict[name]["categories"].append(dir_name)
                        user_dict[name]["book_count"] += 1
                    else:
                        # Xử lý trường hợp liked_count và review_count không có giá trị
                        try:
                            liked_count = int(reviewer["liked_count"].split(" ")[0])
                        except (KeyError, ValueError, IndexError):
                            liked_count = 0

                        try:
                            review_count = int(reviewer["review_count"].split(" ")[0])
                        except (KeyError, ValueError, IndexError):
                            review_count = 0

                        # Lưu trữ dữ liệu người dùng vào từ điển
                        user_dict[name] = {
                            "id": user_id_counter,
                            "name": name,
                            "date": reviewer["date"],
                            "review_count": review_count,
                            "liked_count": liked_count,
                            "categories": [dir_name],
                            "book_count": 1,  # Khởi tạo book_count là 1
                        }

                        # Tăng biến đếm ID người dùng
                        user_id_counter += 1

# Ghi dữ liệu người dùng vào tệp tin JSON
user_data = list(user_dict.values())
output_file = os.path.join(output_dir, "user_data.json")
with open(output_file, "w", encoding="utf-8") as jsonfile:
    json.dump(user_data, jsonfile, ensure_ascii=False, indent=4)

In [10]:
# Tạo biến đếm ID người dùng
user_id_counter = 1

# Tạo một danh sách để lưu trữ dữ liệu người dùng
user_data = []

# Tạo một từ điển để lưu trữ thông tin người dùng dựa trên tên và ngày tham gia
user_dict = {}

# Duyệt qua tất cả các thư mục con trong thư mục gốc
for dir_name in os.listdir(root_dir):
    dir_path = os.path.join(root_dir, dir_name)
    if os.path.isdir(dir_path):
        # Duyệt qua tất cả các tệp tin trong thư mục con
        for file_name in os.listdir(dir_path):
            file_path = os.path.join(dir_path, file_name)
            if file_name.endswith(".json"):
                # Đọc dữ liệu từ tệp tin JSON
                with open(file_path, "r", encoding="utf-8") as f:
                    data = json.load(f)

                # Lấy thông tin người dùng từ dữ liệu
                for review in data["reviews"]:
                    reviewer = review["reviewer"]
                    name = reviewer["name"]
                    date = reviewer["date"]

                    # Kiểm tra xem người dùng đã tồn tại trong từ điển hay chưa
                    user_key = (name, date)
                    if user_key not in user_dict:
                        # Xử lý trường hợp liked_count và review_count không có giá trị
                        try:
                            liked_count = int(reviewer["liked_count"].split(" ")[0])
                        except (KeyError, ValueError, IndexError):
                            liked_count = 0

                        try:
                            review_count = int(reviewer["review_count"].split(" ")[0])
                        except (KeyError, ValueError, IndexError):
                            review_count = 0

                        # Lưu trữ dữ liệu người dùng vào từ điển
                        user_dict[user_key] = {
                            "id": user_id_counter,
                            "name": name,
                            "date": date,
                            "review_count": review_count,
                            "liked_count": liked_count,
                            "categories": [dir_name],
                        }

                        # Tăng biến đếm ID người dùng
                        user_id_counter += 1
                    else:
                        # Nếu người dùng đã tồn tại, cập nhật danh mục
                        if dir_name not in user_dict[user_key]["categories"]:
                            user_dict[user_key]["categories"].append(dir_name)

# Ghi dữ liệu người dùng vào tệp tin JSON
user_data = list(user_dict.values())
output_file = os.path.join(output_dir, "user_data_checked_date.json")
with open(output_file, "w", encoding="utf-8") as jsonfile:
    json.dump(user_data, jsonfile, ensure_ascii=False, indent=4)

In [11]:
import pandas as pd

# Đường dẫn đầy đủ của các file JSON
json_file_1 = os.path.join(output_dir, "user_data.json")
json_file_2 = os.path.join(output_dir, "user_data_checked_date.json")

# Đọc dữ liệu từ file JSON
user_data_1 = pd.read_json(json_file_1)
user_data_2 = pd.read_json(json_file_2)

# Chuyển đổi thành DataFrame
df_1 = pd.DataFrame(user_data_1)
df_2 = pd.DataFrame(user_data_2)

# Ghi dữ liệu ra file CSV
csv_file_1 = os.path.join(output_dir, "user_data.csv")
csv_file_2 = os.path.join(output_dir, "user_data_checked_date.csv")
df_1.to_csv(csv_file_1, index=False)
df_2.to_csv(csv_file_2, index=False)

In [13]:
import csv

# Đường dẫn đầy đủ của các file csv
csv_file_1 = os.path.join(output_dir, "user_data_checked_date.csv")
csv_file_2 = os.path.join(output_dir, "user_data.csv")

# Đọc dữ liệu từ file CSV
with open(csv_file_1, "r", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    user_data_all = list(reader)

# Đọc dữ liệu từ file CSV
with open(csv_file_2, "r", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    user_data = list(reader)

print(f"Tổng số người dùng khi kiểm tra thêm trùng ngày tham gia: {len(user_data_all)}")
print(f"Tổng số người dùng khác tên: {len(user_data)}")

Tổng số người dùng khi kiểm tra thêm trùng ngày tham gia: 98019
Tổng số người dùng khác tên: 76024


In [4]:
import pandas as pd

csv_file = os.path.join(output_dir, "user_data.csv")

# Đọc dữ liệu từ file CSV
df = pd.read_csv(csv_file)

# Thống kê số lượng người dùng theo book_count
book_count_stats = df["book_count"].value_counts().reset_index()
book_count_stats.columns = ["book_count", "count"]

# In kết quả
for index, row in book_count_stats.iterrows():
    book_count = row["book_count"]
    count = row["count"]
    print(f"Số lượng người review {book_count} cuốn sách: {count}")

Số lượng người review 1 cuốn sách: 48743
Số lượng người review 2 cuốn sách: 12630
Số lượng người review 3 cuốn sách: 5559
Số lượng người review 4 cuốn sách: 2836
Số lượng người review 5 cuốn sách: 1672
Số lượng người review 6 cuốn sách: 1036
Số lượng người review 7 cuốn sách: 758
Số lượng người review 8 cuốn sách: 505
Số lượng người review 9 cuốn sách: 374
Số lượng người review 10 cuốn sách: 282
Số lượng người review 11 cuốn sách: 220
Số lượng người review 12 cuốn sách: 165
Số lượng người review 13 cuốn sách: 164
Số lượng người review 14 cuốn sách: 128
Số lượng người review 15 cuốn sách: 102
Số lượng người review 16 cuốn sách: 82
Số lượng người review 17 cuốn sách: 72
Số lượng người review 18 cuốn sách: 61
Số lượng người review 19 cuốn sách: 59
Số lượng người review 20 cuốn sách: 51
Số lượng người review 23 cuốn sách: 35
Số lượng người review 22 cuốn sách: 34
Số lượng người review 21 cuốn sách: 33
Số lượng người review 24 cuốn sách: 31
Số lượng người review 25 cuốn sách: 25
Số lượng ng

In [5]:
# Tạo biến đếm ID item
item_id_counter = 1

# Tạo một danh sách để lưu trữ dữ liệu về sách
item_data = []

# Duyệt qua tất cả các thư mục con trong thư mục gốc
for dir_name in os.listdir(root_dir):
    dir_path = os.path.join(root_dir, dir_name)
    if os.path.isdir(dir_path):
        # Duyệt qua tất cả các tệp tin trong thư mục con
        for file_name in os.listdir(dir_path):
            file_path = os.path.join(dir_path, file_name)
            if file_name.endswith(".json"):
                # Lấy ID từ biến đếm
                file_id = str(item_id_counter)

                # Đọc dữ liệu từ tệp tin JSON
                with open(file_path, "r", encoding="utf-8") as f:
                    data = json.load(f)

                # Lấy thông tin về sách từ dữ liệu
                title = data["info"]["title"]
                author = data["info"]["author"]
                details = data["details"]
                rating = data.get("rating", {})
                total_rating = rating.get("total_rating", 0)
                number_of_rating = rating.get("number_of_rating", 0)
                publisher = details.get("Nhà xuất bản", "")
                cover_type = details.get("Loại bìa", "")
                page_count = details.get("Số trang", "")

                # Lưu trữ thông tin sách vào danh sách
                item_data.append(
                    {
                        "id": file_id,
                        "title": title,
                        "author": author,
                        "total_rating": total_rating,
                        "number_of_rating": number_of_rating,
                        "publisher": publisher,
                        "cover_type": cover_type,
                        "page_count": page_count,
                        "categories": [dir_name],
                    }
                )

                # Tăng biến đếm ID item
                item_id_counter += 1

# Ghi dữ liệu sách vào tệp tin JSON
output_file = os.path.join(output_dir, "item_data.json")
with open(output_file, "w", encoding="utf-8") as jsonfile:
    json.dump(item_data, jsonfile, ensure_ascii=False, indent=4)

In [6]:
import pandas as pd

# Đường dẫn đầy đủ của các file JSON
json_file = os.path.join(output_dir, "item_data.json")

# Đọc dữ liệu từ file JSON
item_data = pd.read_json(json_file)

# Chuyển đổi thành DataFrame
df = pd.DataFrame(item_data)

# Ghi dữ liệu ra file CSV
csv_file = os.path.join(output_dir, "item_data.csv")
df.to_csv(csv_file, index=False)

In [7]:
import csv

# Đọc dữ liệu từ file CSV
file = os.path.join(output_dir, "item_data.json")
with open(file, "r", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    item_data = list(reader)

# Hiển thị tổng số lượng item
print(f"Tổng số lượng item: {len(item_data)}")

Tổng số lượng item: 8153


In [8]:
import csv

# Khởi tạo một từ điển để lưu trữ số lượng item cho mỗi thể loại
category_counts = {}

file = os.path.join(output_dir, "item_data.csv")

# Đọc dữ liệu từ file CSV
with open(file, "r", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        categories = row["categories"].split(
            ","
        )  # Chuyển đổi danh sách thể loại thành danh sách các chuỗi
        for category in categories:
            if category in category_counts:
                category_counts[category] += 1
            else:
                category_counts[category] = 1

# Hiển thị số lượng item cho mỗi thể loại
print("Thống kê số lượng item theo từng thể loại:")
for category, count in category_counts.items():
    print(f"{category}: {count}")

Thống kê số lượng item theo từng thể loại:
['bao-hiem']: 12
['chinh-sach-kinh-te-phat-trien']: 40
['khoa-hoc-ky-thuat']: 54
['kinh-te-vi-mo']: 20
['luyen-thi-ielts']: 42
['luyen-thi-toeic']: 41
['ngu-phap-tieng-anh']: 40
['sach-hoc-tieng-anh-khac']: 51
['sach-kien-thuc-tong-hop']: 54
['sach-kinh-te']: 16
['sach-ky-nang-song']: 5
['sach-van-hoc']: 60
['tam-ly-gioi-tinh']: 5
['thue']: 23
['tien-ky-thuat-so']: 38
['tieng-anh-thieu-nhi']: 40
['trinh-tham']: 53
['tu-vung-tieng-anh']: 38


In [9]:
# Tạo một từ điển để lưu trữ ánh xạ tên người dùng sang user_id
name_to_user_id = {}

# Tạo một từ điển để lưu trữ ánh xạ tên sách sang item_id
title_to_item_id = {}

# Đọc dữ liệu từ file JSON user_data
user_data_file = os.path.join(output_dir, "user_data.json")
with open(user_data_file, "r", encoding="utf-8") as jsonfile:
    user_data = json.load(jsonfile)
    for user in user_data:
        name_to_user_id[user["name"]] = user["id"]

# Đọc dữ liệu từ file JSON item_data
item_data_file = os.path.join(output_dir, "item_data.json")
with open(item_data_file, "r", encoding="utf-8") as jsonfile:
    item_data = json.load(jsonfile)
    for item in item_data:
        title_to_item_id[item["title"]] = item["id"]

# Tạo một danh sách để lưu trữ dữ liệu rating
rating_data = []

# Duyệt qua tất cả các thư mục con trong thư mục gốc
for dir_name in os.listdir(root_dir):
    dir_path = os.path.join(root_dir, dir_name)
    if os.path.isdir(dir_path):
        # Duyệt qua tất cả các tệp tin trong thư mục con
        for file_name in os.listdir(dir_path):
            file_path = os.path.join(dir_path, file_name)
            if file_name.endswith(".json"):
                # Đọc dữ liệu từ tệp tin JSON
                with open(file_path, "r", encoding="utf-8") as f:
                    data = json.load(f)
                    info = data["info"]
                    title = info["title"]
                    item_id = title_to_item_id.get(title, "")

                    # Lặp qua tất cả các review và lưu vào danh sách rating_data
                    for review in data["reviews"]:
                        reviewer = review["reviewer"]
                        name = reviewer["name"]
                        user_id = name_to_user_id.get(name, "")
                        rating = review["review"]["rating"]
                        content = review["review"]["content"]
                        rating_data.append(
                            {
                                "user_id": user_id,
                                "item_id": item_id,
                                "rating": rating,
                                "content": content,
                            }
                        )

# Ghi dữ liệu rating vào tệp tin JSON
rating_data_file = os.path.join(output_dir, "rating.json")
with open(rating_data_file, "w", encoding="utf-8") as jsonfile:
    json.dump(rating_data, jsonfile, ensure_ascii=False, indent=4)

print("File rating.json đã được tạo thành công.")

File rating.json đã được tạo thành công.


In [11]:
import json

# Đọc dữ liệu từ file JSON vào danh sách
rating_data_file = os.path.join(output_dir, "rating.json")
rating_data = []
with open(rating_data_file, "r", encoding="utf-8") as jsonfile:
    rating_data = json.load(jsonfile)

# Sắp xếp danh sách theo user_id
rating_data.sort(key=lambda x: x["user_id"])

# Ghi lại dữ liệu đã sắp xếp vào file JSON mới
rating_sorted_file = os.path.join(output_dir, "rating_sorted.json")
with open(rating_sorted_file, "w", encoding="utf-8") as jsonfile:
    json.dump(rating_data, jsonfile, ensure_ascii=False, indent=4)

print("Dữ liệu đã được sắp xếp theo user_id và ghi vào file rating_sorted.json.")

Dữ liệu đã được sắp xếp theo user_id và ghi vào file rating_sorted.json.


In [21]:
import json
import os
import random

# Đọc dữ liệu từ file JSON
rating_data = []
rating_sorted_file = os.path.join(output_dir, "rating_sorted.json")
with open(rating_sorted_file, "r", encoding="utf-8") as jsonfile:
    rating_data = json.load(jsonfile)

# Khởi tạo danh sách cho tập huấn luyện và tập kiểm tra
train_data = []
test_data = []

# Lưu trữ đánh giá theo user_id
user_reviews = {}

for review in rating_data:
    user_id = review["user_id"]
    if user_id not in user_reviews:
        user_reviews[user_id] = []
    user_reviews[user_id].append(review)

# Chia dữ liệu
for user_id, reviews in user_reviews.items():
    if len(reviews) <= 3:
        train_data.extend(reviews)
    else: # chia dữ liệu thành 5:7 cho train, 2:7 cho test
        random.shuffle(reviews)
        num_reviews = len(reviews)
        num_train = (num_reviews * 5) // 7
        train_data.extend(reviews[:num_train])
        test_data.extend(reviews[num_train:])


# Ghi dữ liệu vào các file CSV tương ứng
def write_data_to_csv(data, filename):
    keys = data[0].keys()
    with open(filename, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=keys)
        writer.writeheader()
        writer.writerows(data)


output_train_file = os.path.join(output_dir, "train_data.csv")
output_test_file = os.path.join(output_dir, "test_data.csv")
write_data_to_csv(train_data, output_train_file)
write_data_to_csv(test_data, output_test_file)

print("Dữ liệu đã được chia thành tập huấn luyện và tập kiểm tra.")

Dữ liệu đã được chia thành tập huấn luyện và tập kiểm tra.
